# 🏭 Warehouse Location Problem with FlexQAOA

This notebook demonstrates solving a **Warehouse Location Problem** using Aqarios's FlexQAOA algorithm. The Warehouse Location Problem is a classic facility location optimization challenge that determines which warehouses to open and how to assign customers to minimize total cost. 🎯

The problem formulation considers:
- 🚚 Transportation costs from warehouses to customers
- 💰 Setup costs for opening new warehouses at given locations

## 🔍 Overview

1. **Setup**
2. **Definition of Warehouse Location Problem with Luna Model**
3. **Solving the optimization problem with Luna**
4. **Comparison of FlexQAOA and building blocks**
5. **Inspecting in Luna Dashboard**
6. **Q&A**

## 📦 Imports and Setup

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os
# Add project root to Python path so utils can be imported
if os.getcwd().endswith('notebooks'):
  sys.path.append('..')
else:
  sys.path.append('.')

In [ ]:
from luna_quantum import Model, Variable, Vtype, quicksum
import os
from utils.plotting import solution_to_arrays
from luna_quantum.solve import algorithms
import getpass
from utils.plotting import plot_warehouse_network
from luna_quantum.solve.parameters.algorithms.base_params import (
    LinearQAOAParams,
    ScipyOptimizerParams,
)
from luna_quantum.solve.parameters.algorithms.quantum_gate.flex_qaoa import (
    IndicatorFunctionParams,
    OneHotParams,
    PipelineParams,
    QuadraticPenaltyParams,
)

if "LUNA_API_KEY" not in os.environ:
    # Prompt securely for the key if not already set
    os.environ["LUNA_API_KEY"] = getpass.getpass("Enter your Luna API key: ")

This section imports Luna Quantum's comprehensive optimization toolkit, designed to make quantum computing accessible to business optimization problems. The Luna SDK provides a seamless interface between classical optimization modeling and cutting-edge quantum algorithms, allowing you to leverage both approaches without deep quantum expertise.

The API key provides secure access to Luna's cloud-based quantum infrastructure, which includes access to state-of-the-art quantum computers and simulators as well as classical algorithms and powerful hardware. This ensuring scalable, reliable computation for complex optimization problems.

![LunaSolve Flow Algorithm Selection](plots/LunaSolve.png)

## 📋 Problem Data Definition

Here we define the key components of our warehouse location problem:


### 🏭 1. Warehouses (Potential Facilities)

In [ ]:
facilities = {
    "Munich": {"cost": 50, "capacity": 7},
    "Nuremberg": {"cost": 40, "capacity": 6},
}

We define the potential warehouse facilities with their associated fixed costs and capacity constraints. In real-world scenarios, this could represent distribution centers, manufacturing plants, or service locations. The warehouse location problem is particularly relevant for supply chain optimization, where companies need to balance infrastructure investment with operational efficiency.

### 🏥 2. Hospitals (Customers/Demand Points)

In [ ]:
hospitals = {
    "H2": {"location": "Munich-North", "demand": 4},
    "H3": {"location": "Nuremberg-South", "demand": 3},
    "H4": {"location": "Augsburg-West", "demand": 2},
    "H8": {"location": "Erlangen", "demand": 1},
}

The demand points (hospitals in this healthcare logistics scenario) represent customer locations with specific supply requirements. Each location has both geographic coordinates and demand quantities that must be satisfied. This type of problem structure is common across industries - from retail distribution to emergency services deployment.

### 🚛 3. Transportation Costs

In [ ]:
transport_costs = {
    ("H2", "Munich"): 0.15,
    ("H2", "Nuremberg"): 0.40,
    ("H2", "Augsburg"): 0.30,
    ("H2", "Regensburg"): 0.55,
    ("H3", "Munich"): 0.40,
    ("H3", "Nuremberg"): 0.12,
    ("H3", "Augsburg"): 0.50,
    ("H3", "Regensburg"): 0.35,
    ("H4", "Munich"): 0.45,
    ("H4", "Nuremberg"): 0.18,
    ("H4", "Augsburg"): 0.55,
    ("H4", "Regensburg"): 0.40,
    ("H8", "Munich"): 0.40,
    ("H8", "Nuremberg"): 0.15,
    ("H8", "Augsburg"): 0.45,
    ("H8", "Regensburg"): 0.30,
}

The transportation cost matrix captures the economic relationships between supply sources and demand locations. In practice, these costs could include distance-based shipping, fuel expenses, delivery time premiums, or complex multi-modal transportation scenarios.

In [ ]:
plot_warehouse_network(facilities, hospitals, transport_costs)

This network visualization helps understand the problem structure and potential solutions. Visual analysis is often crucial for validating optimization results and gaining business insights that pure numerical output might miss.

# Modelling the Warehouse Location Problem with AqModels

In [ ]:
depot = "Munich"
m = Model("WarehouseLocationProblem")

# add variables
x = {}
y = {}
with m.environment:
    for f in facilities:
        for h in hospitals:
            x[(f, h)] = Variable(vtype=Vtype.Binary, name=f"x_{f},{h}")
        y[f] = Variable(vtype=Vtype.Binary, name=f"y_{f}")

# add cost to transport goods
m.objective += quicksum(
    x[f, h] * transport_costs[(h, f)] * hospitals[h]["demand"]
    for f in facilities
    for h in hospitals
)

# add cost to open facility
m.objective += quicksum(y[f] * facilities[f]["cost"] for f in facilities)

# each hospital must be delivered
for h in hospitals:
    m.add_constraint(
        quicksum(x[f, h] for f in facilities) == 1, name=f"deliver_hospital_{h}"
    )

# at most usage of capacity
for f in facilities:
    m.add_constraint(
        quicksum(hospitals[h]["demand"] * x[f, h] for h in hospitals)
        <= facilities[f]["capacity"] * y[f],
        name=f"deliver_{f}",
    )

print(m)

Luna's modeling interface follows familiar mathematical optimization syntax, making it intuitive for quantum researchers, operations research practitioners and data scientists. The binary variables x(f,h) determine which hospitals are served by which facilities, while y(f) decides which facilities to open.

The objective function minimizes total costs by balancing facility opening expenses against transportation costs. The constraints ensure every hospital receives service (demand satisfaction) while respecting facility capacity limits.

# Solving the Warehouse Location Problem with LunaSolve

In [ ]:
scip = algorithms.SCIP()
job = scip.run(model=m, name="WLP with Scip")

Luna's unified interface allows you to easily switch between different solving approaches. Here we use SCIP, an open-source classical optimization solver, to establish the optimal solution baseline. This classical baseline is crucial for evaluating the performance of quantum algorithms.

In [ ]:
solution = job.result()
print(solution)

# 🔄 Model Transformation Pipeline (Upcoming Feature)

In [ ]:
from utils.transformation import move_constraints

t_model = move_constraints(m)
print(t_model)

This demonstrates Luna's automatic model transformation capabilities - converting constrained optimization problems into the unconstrained quadratic form required by quantum algorithms like QAOA. The transformation process is mathematically sophisticated but completely transparent to the user.

 Traditional quantum optimization requires manual tuning and adaptation of the algorithms and model, but Luna's intelligent transformation engine handles this complexity automatically.

In [ ]:
alg = algorithms.SimulatedAnnealing(num_reads=100)
job = alg.run(model=t_model, name="WLP with PA")

Luna provides access to various classical heuristics that can serve as stepping stones toward quantum algorithms, allowing you to gradually transition from classical to quantum approaches while maintaining solution quality confidence. Easily switch to another algorithm by replacing the algorithm code line. See our docs https://docs.aqarios.com/algorithms/ for all available algorithms that you can plug in with one line code change.

In [ ]:
print(job.result())

# 🔬 Indepth comparison of different FlexQAOA configurations
We will compare different FlexQAOA configurations with the same base configuration.

In [ ]:
shared_config = {
    "reps": 3,
    "initial_params": LinearQAOAParams(delta_beta=0.2, delta_gamma=0.2),
    "optimizer": ScipyOptimizerParams(method="BFGS"),
}

To ensure fair comparison across FlexQAOA variants, we establish a shared configuration baseline.

📊 Standard QAOA: A FlexQAOA pipeline without advanced constraint handling (effectively the default QAOA algorithm)

In [ ]:
qaoa = algorithms.FlexQAOA(
    **shared_config,
    pipeline=PipelineParams(
        indicator_function=None,
        one_hot=None,
        quadratic_penalty=QuadraticPenaltyParams(),
    ),
)

Standard QAOA serves as our quantum baseline, using traditional penalty methods to handle constraints. While effective, this approach treats all constraints uniformly, introduces many slack variables, and may struggle with complex constraint interactions.


🔢 One-Hot Encoding FlexQAOA: A FlexQAOA with a one-hot encoding of the binary variables

In [ ]:
oh_qaoa = algorithms.FlexQAOA(
    **shared_config,
    pipeline=PipelineParams(
        indicator_function=None,
        one_hot=OneHotParams(),
        quadratic_penalty=QuadraticPenaltyParams(),
    ),
)

One-hot encoding with XY-mixers represents a significant advancement in constraint-aware quantum optimization. Instead of penalizing constraint violations, this approach uses quantum gates that naturally preserve constraint feasibility during the optimization process.

🚩 Indicator Function FlexQAOA: A FlexQAOA with an indicator function for the binary variables

In [ ]:
if_qaoa = algorithms.FlexQAOA(
    **shared_config,
    pipeline=PipelineParams(
        one_hot=None,
        indicator_function=IndicatorFunctionParams(),
        quadratic_penalty=QuadraticPenaltyParams(),
    ),
)

Indicator functions provide another sophisticated constraint handling approach, particularly effective for inequality constraints like capacity limits. This method uses auxiliary variables and specialized quantum operators to encode constraint satisfaction directly into the quantum state evolution.


🎯 Full FlexQAOA: A FlexQAOA pipeline with full functionality enabled

In [ ]:
flex_qaoa = algorithms.FlexQAOA(**shared_config)

Full FlexQAOA represents Luna's most advanced optimization capability - an intelligent pipeline that automatically selects the best constraint handling technique for each constraint type in your problem. This hybrid approach combines one-hot encoding, indicator functions, and penalty methods as needed.

What makes this particularly powerful is Luna's ability to analyze your problem structure and automatically configure the most effective algorithmic approach. This removes the guesswork from quantum algorithm selection and typically delivers superior performance compared to any single-method approach.

In [ ]:
oh_job = oh_qaoa.run(model=m, name="QAOA with Onehots")
if_job = if_qaoa.run(model=m, name="QAOA with Indicators Function")
flex_job = flex_qaoa.run(model=m, name="QAOA with FlexQAOA")
qaoa_job = qaoa.run(model=m, name="QAOA Default")

Luna's cloud infrastructure enables parallel execution of multiple algorithm variants, allowing comprehensive performance comparison without lengthy sequential runs. Each job is automatically queued and executed on appropriate quantum hardware or high-performance simulators.


## ⚡ FlexQAOA Algorithm Configuration

**FlexQAOA** (Flexible Quantum Approximate Optimization Algorithm) is a quantum optimization algorithm that can handle constrained optimization problems efficiently! 🔧

### 🛠️ Algorithm Parameters:
- **reps**: Number of QAOA layers (depth of the quantum circuit) 🔄
- **initial_params**: Starting parameters for the optimization 🎯
- **optimizer**: Classical optimizer used to update QAOA parameters 🚀

In [ ]:
qaoa_sol = qaoa_job.result()
ohc_sol = oh_job.result()
if_sol = if_job.result()
flex_sol = flex_job.result()

Luna's asynchronous job management handles the complexity of quantum computation timing and resource allocation. The platform automatically manages queue positions, hardware availability, and result retrieval, providing a smooth development experience despite the underlying quantum infrastructure complexity.

The result objects provide comprehensive optimization metadata including solution quality distributions, sampling overview, and execution timing and a lot of handy methods like ```.expectation_value()``` or ```.feasibility_ratio()``` - all essential for evaluating quantum algorithm performance in production environments.

## 📥 Solution Retrieval

The algorithm runs asynchronously on quantum hardware or simulators. The `job.result()` method retrieves the solution once the optimization is complete. ⏳

In [ ]:
import matplotlib.pyplot as plt

flex_obj, flex_count = solution_to_arrays(flex_sol)
ohc_obj, ohc_count = solution_to_arrays(ohc_sol)
if_obj, if_count = solution_to_arrays(if_sol)
qaoa_obj, qaoa_count = solution_to_arrays(qaoa_sol)
plt.figure(figsize=(12, 6))
plt.step(
    qaoa_obj, qaoa_count, label="Objective Function QAOA", color="#013CF2", linewidth=3
)
plt.step(
    flex_obj,
    flex_count,
    label="Objective Function FlexQAOA",
    color="#EBC25B",
    linewidth=3,
)
plt.step(
    ohc_obj,
    ohc_count,
    label="Objective Function XY-Mixer QAOA",
    color="#538A6A",
    linewidth=3,
)
plt.step(
    if_obj,
    if_count,
    label="Objective Function Indicator QAOA",
    color="#7F7F7F",
    linewidth=3,
)

# Add labels and title
plt.xlabel("Objective Value")
plt.ylabel("Percentage of Samples")
plt.title("Feasible solution")
plt.ylim(0, 1)

plt.grid(False)
plt.axvline(
    x=91.47, color="black", linestyle="--", linewidth=2, label="Optimal Solution"
)

# Position legend to the right outside the plot area
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")

# Adjust layout to prevent legend cutoff
plt.tight_layout()

# Show the plot
plt.show()

This comparative analysis reveals the practical advantages of Luna's advanced FlexQAOA techniques. The plot shows how different constraint handling approaches affect solution quality distribution - a critical metric for evaluating quantum optimization performance.

![LunaSolve Flow Algorithm Selection](plots/LunaDashboard.png)

## 🎉 Conclusion

This notebook demonstrated how to solve a **Warehouse Location Problem** using Luna Quantum's FlexQAOA algorithm! 🚀

### 🔑 Key Takeaways:

1. **Problem Modeling**: Easily model with Luna Model library, load existing models with translators or use our predefined use cases
2. **Quantum and Classical Algorithms**: Easily switch between algorithms by leveraging our intuitive interface and transformations  
3. **Solution Quality**: FlexQAOA applies the most efficient optimization pipeline to find better solutions faster

**Happy optimizing!** 🎯✨

![LunaSolve Flow Algorithm Selection](plots/ApplicationsAreas.png)